In [1]:
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
import numpy as np

from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
# m1 = pd.read_csv("WhoScored_ENG-Premier League.csv")
m2 = pd.read_csv("WhoScored_ESP-La Liga.csv")
# m3 = pd.read_csv("WhoScored_FRA-Ligue 1.csv")
# m4 = pd.read_csv("WhoScored_GER-Bundesliga.csv")
# m5 = pd.read_csv("WhoScored_ITA-Serie A.csv")


In [6]:
matches = pd.concat([m2], axis = 0)

In [7]:
matches = matches[["game","game_id","type","outcome_type","team","fecha","player", "x", "y"]]
matches = matches.sort_values(by=["fecha","game_id"], ignore_index=True)

In [5]:
example_match = matches[matches["game_id"] == 1729483] # 1729340  1729483 615168

In [5]:
example_match

NameError: name 'example_match' is not defined

In [7]:
print(example_match["type"].unique())
print(example_match["outcome_type"].unique())

['FormationSet' 'Start' 'Pass' 'End' 'BallTouch' 'Foul' 'Aerial'
 'Interception' 'Dispossessed' 'Tackle' 'TakeOn' 'Clearance'
 'CornerAwarded' 'Claim' 'BlockedPass' 'Challenge' 'BallRecovery'
 'SavedShot' 'Save' 'KeeperPickup' 'OffsidePass' 'OffsideProvoked'
 'OffsideGiven' 'MissedShots' 'KeeperSweeper' 'ShieldBallOpp'
 'SubstitutionOff' 'SubstitutionOn' 'Goal' 'Punch' 'FormationChange'
 'CrossNotClaimed' 'Card']
['Successful' 'Unsuccessful']


In [8]:
example_match.game_id.unique()[0]

1729483

In [8]:
def get_features_individuales(df_matches, match, n_matches, bins_x, bins_y):
    
    ## Features elegidos
    features = ['Pass', 'TakeOn', 'Tackle', 'BallRecovery'
                'Interception', 'BlockedPass', 'Clearance', 'CornerAwarded', 'Aerial', 'Foul',
                'Dispossessed', 'BallTouch', 'ShieldBallOpp', 'Challenge', 'MissedShots',
                'SavedShot', 'Save', 'KeeperPickup', 'Goal', 'KeeperSweeper', 'Card', 'Smother',
                'OffsideProvoked', 'OffsidePass', 'OffsideGiven', 'Punch', 'Error']
    ##
    date = match["fecha"].unique()[0]

    df_matches_limited = df_matches[df_matches["fecha"] < date]
    str_teams = match["team"].unique()
    str_team1 = str_teams[0]
    str_team2 = str_teams[1]
    
    #### SELECCIONAR LOS 10 CON MAYOR PARTICIPACION + PORTERO
    team1 = []
    team2 = []
    ## Seleccionar portero
    keeperPickup = match[match["type"]== "KeeperPickup"]
    keeper1 = keeperPickup[keeperPickup["team"] == str_team1].player.unique()
    keeper2 = keeperPickup[keeperPickup["team"] == str_team2].player.unique()

    if len(keeper1) > 0:
        team1.append(keeper1[0])
        no_keeper_team1 = match[match["team"] == str_team1]["player"].value_counts().drop(team1[0]).index[:11-len(team1)].to_list()
    if len(keeper2) > 0:
        team2.append(keeper2[0])
        no_keeper_team2 = match[match["team"] == str_team2]["player"].value_counts().drop(team2[0]).index[:11-len(team2)].to_list()
    # Si no se detecta un portero, agregar un vacio
    if  len(team1) == 0:
        no_keeper_team1 = match[match["team"] == str_team1]["player"].value_counts().index[:11-len(team1)].to_list()
        
    if  len(team2) == 0:
        no_keeper_team2 = match[match["team"] == str_team2]["player"].value_counts().index[:11-len(team2)].to_list()
        
    team1.extend(no_keeper_team1)
    team2.extend(no_keeper_team2)
    ####

    # Features team1
    all_players = []
    for player in team1:
        dict_player = {"Player": player}
        ids = df_matches_limited[df_matches_limited["player"] == player].game_id.unique()[-n_matches:]
        player_events = df_matches_limited[df_matches_limited["player"] == player]
        player_events = player_events[player_events["game_id"].isin(ids)]

        pitch = Pitch(pitch_type='opta', pitch_color='white',
                      line_color='black', line_zorder=2)

        for fea in features:
            event_succesful = player_events[(player_events["type"] == fea) & (
                player_events["outcome_type"] == 'Successful')]
            event_unsuccessful = player_events[(player_events["type"] == fea) & (
                player_events["outcome_type"] == 'Unsuccessful')]

            bin_statistic_succesful = pitch.bin_statistic(
                event_succesful.x, event_succesful.y, statistic="count", bins=(bins_x, bins_y), normalize=False)

            bin_statistic_unsuccesful = pitch.bin_statistic(
                event_unsuccessful.x, event_unsuccessful.y, statistic="count", bins=(bins_x, bins_y), normalize=False)

            bins_successful = bin_statistic_succesful["statistic"].flatten()
            bins_unsuccessful = bin_statistic_unsuccesful["statistic"].flatten()

            for i in range(len(bins_successful)):
                dict_player[f"{fea}_Successful_bin_{i}"] = bins_successful[i]
                dict_player[f"{fea}_Unsuccessful_bin_{i}"] = bins_unsuccessful[i]
            # Agregar la cantidad de juegos 
            dict_player["Number_matches"] = len(player_events["game"].unique())

        all_players.append(dict_player)

    df_all_players_team1 = pd.DataFrame(all_players)

    # Features team2
    all_players = []
    for player in team2:
        dict_player = {"Player": player}
        ids = df_matches_limited[df_matches_limited["player"] == player].game_id.unique()[-n_matches:]
        player_events = df_matches_limited[df_matches_limited["player"] == player]
        player_events = player_events[player_events["game_id"].isin(ids)]


        pitch = Pitch(pitch_type='opta', pitch_color='white',
                      line_color='black', line_zorder=2)

        for fea in features:
            event_succesful = player_events[(player_events["type"] == fea) & (
                player_events["outcome_type"] == 'Successful')]
            event_unsuccessful = player_events[(player_events["type"] == fea) & (
                player_events["outcome_type"] == 'Unsuccessful')]

            bin_statistic_succesful = pitch.bin_statistic(
                event_succesful.x, event_succesful.y, statistic="count", bins=(bins_x, bins_y), normalize=False)

            bin_statistic_unsuccesful = pitch.bin_statistic(
                event_unsuccessful.x, event_unsuccessful.y, statistic="count", bins=(bins_x, bins_y), normalize=False)

            bins_successful = bin_statistic_succesful["statistic"].flatten()
            bins_unsuccessful = bin_statistic_unsuccesful["statistic"].flatten()

            for i in range(len(bins_successful)):
                dict_player[f"{fea}_Successful_bin_{i}"] = bins_successful[i]
                dict_player[f"{fea}_Unsuccessful_bin_{i}"] = bins_unsuccessful[i]
            # Agregar la cantidad de juegos 
            dict_player["Number_matches"] = len(player_events["game"].unique())

        all_players.append(dict_player)
    df_all_players_team2 = pd.DataFrame(all_players)

    # Agrgar el nombre del equipo
    df_all_players_team1["Team"] = str_team1 
    df_all_players_team2["Team"] = str_team2

    # Agregar fecha
    df_all_players_team1["Date"] = date
    df_all_players_team2["Date"] = date
    
    # Agregar id
    df_all_players_team1["game_id"] = match.game_id.unique()[0]
    df_all_players_team2["game_id"] = match.game_id.unique()[0]

    # Agregar resultado goles del equipo final
    df_all_players_team1["Goals"] = len(match[(match["type"] == "Goal") & (match["team"] == str_team1)])
    df_all_players_team2["Goals"] = len(match[(match["type"] == "Goal") & (match["team"] == str_team2)])

    columnas_a_mover = ["Player","Team","Date","Goals","Number_matches","game_id"]

    # Obtén una lista de todas las columnas en el DataFrame
    columnas = df_all_players_team1.columns.tolist()

    # Mueve las columnas deseadas al principio
    for columna in reversed(columnas_a_mover):
        columnas.insert(0, columnas.pop(columnas.index(columna)))
    # Reordena las columnas del DataFrame
    df_all_players_team1 = df_all_players_team1[columnas]
    df_all_players_team2 = df_all_players_team2[columnas]

    return df_all_players_team1, df_all_players_team2

df_all_players_team1, df_all_players_team2 = get_features_individuales(matches, example_match, n_matches = 15, bins_x = 2, bins_y = 2)

NameError: name 'example_match' is not defined

In [10]:
df_all_players_team1

,Player,Team,Date,Goals,Number_matches,game_id,Pass_Successful_bin_0,Pass_Unsuccessful_bin_0,Pass_Successful_bin_1,Pass_Unsuccessful_bin_1,...,Punch_Successful_bin_3,Punch_Unsuccessful_bin_3,Error_Successful_bin_0,Error_Unsuccessful_bin_0,Error_Successful_bin_1,Error_Unsuccessful_bin_1,Error_Successful_bin_2,Error_Unsuccessful_bin_2,Error_Successful_bin_3,Error_Unsuccessful_bin_3
0,Jordan Pickford,Everton,2024-02-19,1,15,1729483,202.0,127.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,James Tarkowski,Everton,2024-02-19,1,15,1729483,62.0,26.0,28.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,Ben Godfrey,Everton,2024-02-19,1,15,1729483,33.0,35.0,34.0,19.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Vitalii Mykolenko,Everton,2024-02-19,1,15,1729483,191.0,74.0,201.0,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,James Garner,Everton,2024-02-19,1,15,1729483,86.0,29.0,135.0,53.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Jarrad Branthwaite,Everton,2024-02-19,1,15,1729483,286.0,79.0,39.0,9.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Dominic Calvert-Lewin,Everton,2024-02-19,1,15,1729483,13.0,24.0,37.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Idrissa Gueye,Everton,2024-02-19,1,15,1729483,84.0,15.0,99.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Dwight McNeil,Everton,2024-02-19,1,15,1729483,104.0,34.0,171.0,97.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Ashley Young,Everton,2024-02-19,1,15,1729483,26.0,17.0,39.0,15.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_all_players_team2

,Player,Team,Date,Goals,Number_matches,game_id,Pass_Successful_bin_0,Pass_Unsuccessful_bin_0,Pass_Successful_bin_1,Pass_Unsuccessful_bin_1,...,Punch_Successful_bin_3,Punch_Unsuccessful_bin_3,Error_Successful_bin_0,Error_Unsuccessful_bin_0,Error_Successful_bin_1,Error_Unsuccessful_bin_1,Error_Successful_bin_2,Error_Unsuccessful_bin_2,Error_Successful_bin_3,Error_Unsuccessful_bin_3
0,Sam Johnstone,Crystal Palace,2024-02-19,1,15,1729483,97.0,77.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Tyrick Mitchell,Crystal Palace,2024-02-19,1,15,1729483,213.0,106.0,209.0,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Jordan Ayew,Crystal Palace,2024-02-19,1,15,1729483,42.0,16.0,155.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Joachim Andersen,Crystal Palace,2024-02-19,1,15,1729483,142.0,39.0,34.0,14.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Adam Wharton,Crystal Palace,2024-02-19,1,2,1729483,13.0,5.0,9.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Chris Richards,Crystal Palace,2024-02-19,1,15,1729483,110.0,25.0,49.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Daniel Muñoz,Crystal Palace,2024-02-19,1,2,1729483,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Jean-Philippe Mateta,Crystal Palace,2024-02-19,1,15,1729483,15.0,6.0,50.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Joel Ward,Crystal Palace,2024-02-19,1,15,1729483,1.0,2.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,Odsonne Édouard,Crystal Palace,2024-02-19,1,15,1729483,21.0,4.0,69.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
n_matches = 15
bins_x = 12
bins_y = 8
n_workers = 12
matches_id = matches["game_id"].unique()
resultado = []

#En serie 
for id in tqdm(matches_id):
    data = get_features_individuales(matches, matches[matches["game_id"] == id], n_matches, bins_x, bins_y)
    resultado.append(data)

# #En paralelo
# resultado = Parallel(n_jobs=n_workers, verbose=10)(
#     delayed(get_features_individuales)(matches[matches["fecha"]<=matches[matches["game_id"] == id]["fecha"].unique()[0]], matches[matches["game_id"] == id], n_matches, bins_x, bins_y) for id in tqdm(matches_id))
    
dfs_list = []
for df in resultado:
    dfs_list.append(df[0])
    dfs_list.append(df[1])
df = pd.concat(dfs_list, axis = 0)
df.to_csv(f"ESP-La LigaWhoScoredNumMatches{n_matches}Spatial{bins_x}x{bins_y}.csv")

100%|██████████| 4810/4810 [8:56:53<00:00,  6.70s/it]  
